<a href="https://colab.research.google.com/github/RahaviSelvarajan/yolov5_colab/blob/main/SBI_YOLO_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!python --version

##***Mount Google Drive***

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##***Mount Onedrive to get the dataset***

In [ ]:
!wget https://downloads.rclone.org/v1.57.0/rclone-v1.57.0-linux-amd64.deb
!apt install ./rclone-v1.57.0-linux-amd64.deb

In [ ]:
!rclone config

In [ ]:
!sudo mkdir /content/onedrive
!nohup rclone --vfs-cache-mode writes mount onedrive: /content/onedrive &

**Run the below cell only when onedrive gets disconnected in the middle of training and mount it again using the above command.**

In [ ]:
!fusermount -u /content/onedrive

##***Git Clone YOLOv5-Pytorch model***

In [1]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 12111, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 12111 (delta 6), reused 7 (delta 3), pack-reused 12093
Receiving objects: 100% (12111/12111), 11.94 MiB | 17.15 MiB/s, done.
Resolving deltas: 100% (8380/8380), done.


In [ ]:
!python detect.py --source '/content/outpy.avi'

In [ ]:
!unzip /content/drive/MyDrive/val.zip

Install the required python packages

In [3]:
%cd yolov5
!pip install -r requirements.txt

/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 29.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


##***Training***

In [12]:
# Copy data configuration file into yolo directory
!cp /content/drive/MyDrive/Yolov5/sleepaid.yaml /content/yolov5/data/

In [5]:
# Import libraries
import os
import glob
import shutil
from pathlib import Path

In [14]:
val_imgs = Path('/content/yolov5/data/images/test')
val_labels = Path('/content/yolov5/data/labels/test')

if val_imgs.exists():
  shutil.rmtree(val_imgs)
if val_labels.exists():
  shutil.rmtree(val_labels)

os.mkdir(val_imgs)
os.makedirs(val_labels)  

for folder in os.listdir('/content/val'):
  folder_path = os.path.join('/content/val', folder)
  if not folder.startswith('.'):
    for file in os.listdir(folder_path):
      extension = os.path.splitext(file)[1]
      f_path = os.path.join('/content/val',folder, file)
      if extension == '.txt':
        shutil.copy(f_path, val_labels)
      elif extension == '.jpg':
        shutil.copy(f_path, val_imgs)

In [ ]:
!python val.py --batch-size 8 --img-size 1920 --data 'sleepaid.yaml' --weights '/content/drive/MyDrive/Yolov5/runs/final_train/results2/weights/best.pt' --project '/content/drive/MyDrive/Yolov5/runs/final_train' 

In [ ]:
# Get the train and test dataset from different folds
def train_test_split(root_dir, test_fold):

    # Make directory for training and testing images
    train_imgs = Path('/content/yolov5/data/images/train')
    test_imgs = Path('/content/yolov5/data/images/test')

    if train_imgs.exists():
      shutil.rmtree(train_imgs)
    if test_imgs.exists():
      shutil.rmtree(test_imgs)

    os.mkdir(train_imgs)
    os.mkdir(test_imgs)

    # Make directory for target labels
    train_labels = Path('/content/yolov5/data/labels/train')
    test_labels = Path('/content/yolov5/data/labels/test')

    if train_labels.exists():
      shutil.rmtree(train_labels)
    if test_labels.exists():
      shutil.rmtree(test_labels)

    os.makedirs(train_labels)
    os.makedirs(test_labels)

    for j, image in enumerate(glob.iglob(os.path.join('/content/drive/MyDrive/Negatives', "*.jpg"))):
      if j<150:
        shutil.copy(image, train_imgs)
      else:
        shutil.copy(image, test_imgs)

    for i, folder in enumerate(os.listdir(root_dir)):
      folder_path = os.path.join(root_dir, folder)
      file_list = [image for image in glob.glob(os.path.join(folder_path, '*.jpg'))]

      # Get the train folds
      if i != test_fold:
        for image in file_list:
          shutil.copy(image, train_imgs)
          shutil.copy(os.path.splitext(image)[0] + '.txt', train_labels)

      # Get the test fold
      elif i == test_fold:
        for image in file_list:
          shutil.copy(image, test_imgs)
          shutil.copy(os.path.splitext(image)[0] + '.txt', test_labels)

for i in range(1):
  train_test_split('/content/drive/MyDrive/Dataset', i)
  !python train.py --batch-size 8 --img-size 1920 --data 'sleepaid.yaml' --weights 'yolov5s.pt' --epochs 300 --cfg 'yolov5s.yaml'  \
                   --save-period 50 --patience 20 --hyp '/content/drive/MyDrive/Yolov5/hyp.yaml' --project '/content/drive/MyDrive/Yolov5/runs/final_train' 
  !mv /content/drive/MyDrive/Yolov5/runs/final_train/exp /content/drive/MyDrive/Yolov5/runs/final_train/results"$i"